# Demonstration of using Curve registry

### Essential links:
* Source code: <https://github.com/curvefi/curve-pool-registry/tree/b17>;
* ABI: <https://github.com/curvefi/curve-pool-registry/blob/b17/deployed/2020-06-20/registry.abi>;
* Registry contract: `0x7002B727Ef8F5571Cb5F9D70D13DBEEb4dFAe9d1`.

### Complimentary information for tests:
* Y pool: `0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51` (underlying coins: DAI/USDC/USDT/TUSD, compounding coins: yDAI/yUSDC/yUSDT/yTUSD);
* DAI: `0x6B175474E89094C44Da98b954EedeAC495271d0F`;
* USDT: `0xdAC17F958D2ee523a2206206994597C13D831ec7`;

In [1]:
# Init Brownie environment
# <https://eth-brownie.readthedocs.io/en/stable/python-package.html#accessing-the-network>
from brownie import network, Contract
network.connect('mainnet')

In [2]:
import json
with open('registry.abi', 'r') as f:
    abi = json.load(f)
registry = Contract.from_abi('CurveRegistry', '0x7002B727Ef8F5571Cb5F9D70D13DBEEb4dFAe9d1', abi)

### Finding a pool by coins
Pools can be found for a given pair (from -> to)

In [3]:
registry.find_pool_for_coins("0xdAC17F958D2ee523a2206206994597C13D831ec7",
                             "0x6B175474E89094C44Da98b954EedeAC495271d0F", 0)

'0x52EA46506B9CC5Ef470C5bf89f17Dc28bB35D85C'

In [4]:
registry.find_pool_for_coins("0xdAC17F958D2ee523a2206206994597C13D831ec7",
                             "0x6B175474E89094C44Da98b954EedeAC495271d0F", 1)

'0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51'

In [5]:
registry.find_pool_for_coins("0xdAC17F958D2ee523a2206206994597C13D831ec7",
                             "0x6B175474E89094C44Da98b954EedeAC495271d0F", 10)  # ... eventually we hit 0x0000

'0x0000000000000000000000000000000000000000'

### Getting pool's coins
Information is given in format:
```python
[
    (compounding_coin, compounding_coin, ...),
    (underlying_coin, underlying_coin, ...),
    (compounding_coin_decimals, compounding_coin_decimals, ...),
    (underlying_coin_decimals, underlying_coin_decimals, ...)
]
```

In [6]:
registry.get_pool_coins('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51')

(('0x16de59092dAE5CcF4A1E6439D611fd0653f0Bd01',
  '0xd6aD7a6750A7593E092a9B218d66C0A814a3436e',
  '0x83f798e925BcD4017Eb265844FDDAbb448f1707D',
  '0x73a052500105205d34Daf004eAb301916DA8190f',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000'),
 ('0x6B175474E89094C44Da98b954EedeAC495271d0F',
  '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
  '0xdAC17F958D2ee523a2206206994597C13D831ec7',
  '0x0000000000085d4780B73119b644AE5ecd22b376',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000',
  '0x0000000000000000000000000000000000000000'),
 (18, 6, 6, 18, 0, 0, 0, 0),
 (18, 6, 6, 18, 0, 0, 0, 0))

### Pool info
Format:
```python
[
    (balance_0, balance_1, ...),
    (underlying_0, underlying_1, ...),
    (*precisions_for_compounding_coins ...),
    (*precisions_for_underlying_coins ...),
    pool_token_address,
    amplification,
    fee multiplied by 1e10
]
```

In [8]:
registry.get_pool_info('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51')

((469392802249827477767361,
  2309822460525,
  6407926209046,
  1440922583819273636840363,
  0,
  0,
  0,
  0),
 (477754514237630539249784,
  2669612748083,
  6564872766170,
  1461194627546448822416580,
  0,
  0,
  0,
  0),
 (18, 6, 6, 18, 0, 0, 0, 0),
 (18, 6, 6, 18, 0, 0, 0, 0),
 '0xdF5e0e81Dff6FAF3A7e52BA697820c5e32D806A8',
 2000,
 4000000)

With these rates, underlying_balance = balance * rate / 1e18

In [9]:
registry.get_pool_rates('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51')

(1017813890514990603,
 1155765342881459687,
 1024492566238266075,
 1014068794503478882,
 0,
 0,
 0,
 0)

### Estimate the most probable gas spent for exchanging two coins

In [10]:
registry.estimate_gas_used('0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                           '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                           '0x6B175474E89094C44Da98b954EedeAC495271d0F')

799000

### Calculate exchange amount(s)
We're swapping USDC (6 digits) to DAI (18 digits)

In [11]:
dai_amount = registry.get_exchange_amount(
                             '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                             '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                             '0x6B175474E89094C44Da98b954EedeAC495271d0F',
                             10 ** 6)  # Dump 1 USDC
dai_amount / 1e18  # DAI has 18 digits

0.9863416300748085

How much USDC do we need to get 1 DAI?

In [12]:
usdc_amount = registry.get_input_amount(
                             '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                             '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                             '0x6B175474E89094C44Da98b954EedeAC495271d0F',
                             10 ** 18)
usdc_amount / 1e6

1.013847

Get many exchange amounts (DAI for USDC) at once

In [16]:
amounts = registry.get_exchange_amounts(
                             '0x45F783CCE6B7FF23B2ab2D70e416cdb7D6055f51',
                             '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                             '0x6B175474E89094C44Da98b954EedeAC495271d0F',
                             [x * 10 ** 6 for x in range(1, 101)])
[x / 1e18 for x in amounts][:5]  # Let's show only first 5 out of 100

[0.9909413097086051,
 1.9818825872688577,
 2.9728238326805827,
 3.963765045943605,
 4.95470622705775]

Registry can also do `exchange()`, - the demo will be updated with that, too